<a href="https://colab.research.google.com/github/shaem454/Coursera_Capstone/blob/master/Sushi_NZ_Cities_Vs_Moreland.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [188]:
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
!pip install lxml

import folium # map rendering library

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [189]:
CLIENT_ID = '2WEIVPXNKBQZB1W2O220DXKIIMIMVOJQLG2NXSZZYO4KJI3L' # your Foursquare ID
CLIENT_SECRET = 'T' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2WEIVPXNKBQZB1W2O220DXKIIMIMVOJQLG2NXSZZYO4KJI3L
CLIENT_SECRET:T


In [0]:
LIMIT = 500 # Maximum is 100
cities = ['Brunswick VIC','Auckland NZ', 'Wellington NZ', 'Christchurch NZ', 'Hamilton NZ', 'Tauranga NZ']
radius = 5000
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&radius={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        radius,
        "4bf58dd8d48988d1d2941735") # Sushi PLACE CATEGORY ID
    results[city] = requests.get(url).json()


In [0]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [192]:

maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of sushi places in {city} = ", results[city]['response']['totalResults'])


Total number of sushi places in Brunswick VIC =  127
Total number of sushi places in Auckland NZ =  96
Total number of sushi places in Wellington NZ =  37
Total number of sushi places in Christchurch NZ =  26
Total number of sushi places in Hamilton NZ =  19
Total number of sushi places in Tauranga NZ =  17


In [193]:
maps[cities[0]]

In [194]:
maps[cities[1]]

In [195]:
maps[cities[2]]

In [196]:
maps[cities[3]]

In [197]:
maps[cities[4]]

In [198]:
maps[cities[5]]